# Assignment 1: Review of Markov Decision Processes

**Note to students on how this assignment is organized.**
Your answers should be written in a file called `hw1.py` that is located in the same directory as this file. (OR, it can be in a different directory, but the directory should be on your `PYTHONPATH`, so `import hw1` succeeds.)

If you look at `hw1_template.py`, you can see all of the functions you have to write and values you have to set. You won't have to write modify this ipython notebook.


This assignment will review exact methods for solving Markov Decision Processes (MDPs) with finite state and action spaces.
We will implement policy iteration (PI) and value iteration (VI) for a finite MDP, both of which find the optimal policy in a finite number of iterations.

For this assignment, we will consider discounted infinite-horizon MDPs. Here, the MDP is defined by the tuple $(S, A, R, P, \gamma)$, where

- S: state space (set)
- A: action space (set)
- R(s,a,s'): reward function, $S \times A \times S \rightarrow \mathbb{R}$, where $s$ is current state and $s'$ is next state 
- P(s,a,s'): transition probability kernel $p(s' | s, a)$, $S \times A \times S \rightarrow \mathbb{R}$
- $\gamma$: discount $\in (0,1)$

Here we will consider MDPs where $S,A$ are finite sets, hence $R$ and $P$ are 3D arrays

Next, we'll randomly generate an MDP which your algorithms should be able to solve.
Using randomly generated MDPs might seem a bit dry (don't worry, we'll look at some exciting ones later!), but it emphasizes that policy iteration and value iteration just boil down to a few operations on arrays, when we have finite state and action spaces.

In [17]:
# == Notebook setup ==
%load_ext autoreload
%autoreload 2
INSTRUCTOR=True # this flag switches between instructor mode and student mode

import numpy as np, numpy.random as nr
np.random.seed(0)

import hw_utils
import hw1 # YOUR ANSWERS HERE
# ====================

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Part 1: Value Iteration

First, let's randomly generate the reward function and transition probabilities.

In [18]:
nS_rand = 10 # number of states
nA_rand = 2 # number of actions
R_rand = nr.rand(nS_rand, nA_rand, nS_rand) # reward function
# R[i,j,k] := R(s=i, a=j, s'=k), 
# i.e., the dimensions are (current state, action, next state)
P_rand = nr.rand(nS_rand, nA_rand, nS_rand) 
# P[i,j,k] := P(s'=k | s=i, a=j)
# i.e., dimensions are (current state, action, next state)

# BE CAREFUL THAT YOU DON'T MIX UP THE 0TH AND 2ND DIMENISION OF R AND T!
# REMEMBER THAT THE AXES CORRESPOND TO S,A,S', NOT S',A,S
P_rand /= P_rand.sum(axis=2,keepdims=True) # normalize conditional probabilities
gamma = 0.90

### Problem 1a: implement value iteration update
Your task is to write a function called `vstar_backup`, taking the following arguments:

- Vprev: value function from previous iteration $V^{(n)}$
- T: transition function
- R: reward function
- gamma: discount factor.

It should return the following arrays:

- V: new value function $V^{(n+1)}$, defined as follows:

$$V^{(n+1)}(s) = \max_a \sum_{s'} P(s,a,s') [ R(s,a,s') + \gamma V^{(n)}(s')]$$

- $A^{(n+1)}$: the actions that are greedy with respect to $V^{(n)}$, i.e.,

$$V^{(n+1)}(s) = \operatorname*{argmax}_a \sum_{s'} P(s,a,s') [ R(s,a,s') + \gamma V^{(n)}(s')]$$


This update is often called a **backup**, since we are updating the state $s$ based on possible  future states $s'$, i.e., we are propagating the value function *backwards in time*. 
The function is called **vstar**_backup because the fixed point of this update is the optimal value function $V^*$.

Your function will be called by the value iteration loop, provided below.
Your function's output should identically match the provided output.

In [19]:
def value_iteration(T,R,gamma, n_iter):
    """
    Run `n_iter` iterations of value iteration, where T,R,gamma are the
    transition probabilities, reward functions, and discount factor
    of an MDP.
    """
    nS = T.shape[0]
    Vprev = np.zeros(nS)
    Aprev = None
    print hw_utils.fmt_row(13, ["iter", "max|V-Vprev|", "# chg actions", "V[0]"], header=True)
    for i in xrange(n_iter):
        V,A = hw1.vstar_backup(Vprev, T, R, gamma)    
        print hw_utils.fmt_row(13, [i+1, np.abs(V-Vprev).max(), "N/A" if Aprev is None else (A != Aprev).sum(), V[0]])
        Vprev,Aprev = V,A
        
value_iteration(T_rand,R_rand,gamma, n_iter=20)

         iter |  max|V-Vprev| | # chg actions |          V[0]
-------------------------------------------------------------
            1 |      0.707147 |           N/A |      0.618258
            2 |      0.514599 |             1 |       1.13286
            3 |      0.452404 |             0 |       1.58322
            4 |      0.405723 |             0 |       1.98855
            5 |      0.364829 |             0 |       2.35327
            6 |      0.328307 |             0 |       2.68157
            7 |      0.295474 |             0 |       2.97704
            8 |      0.265926 |             0 |       3.24297
            9 |      0.239333 |             0 |        3.4823
           10 |        0.2154 |             0 |        3.6977
           11 |       0.19386 |             0 |       3.89156
           12 |      0.174474 |             0 |       4.06604
           13 |      0.157026 |             0 |       4.22306
           14 |      0.141324 |             0 |       4.36439
        

You'll notice that value iteration only takes two iterations to converge to the right actions everywhere. (However, note that the actual values converge rather slowly.) Think about why that might be the case for MDPs with transition probabilities generated as above.
Also, note that the value of any particular state (e.g., V[0], shown in the rightmost column) increases monotonically. Under which conditions that is true? [question will not be graded]

### Problem 1b: create an MDP such for which value iteration takes a long time to converge
Specifically, the requirement is that your MDP should have 10 states and 2 actions, and at least one element of $A^{(n)}$ should change each iteration (i.e., `#change actions` in the displayed table should be nonzero for at least iterations 2 through 9).

Here's a hint for one solution: arrange the states a line, so that state i can only transition to one of {i-1, i, i+1}.
You should create 3D arrays T,R in `hw1.py` that define the MDP.

In [20]:
nS = 10
assert hw1.P.shape == (10,2,10), "T has the wrong shape"
assert hw1.R.shape == (10,2,10), "R has the wrong shape"
assert np.allclose(hw1.P.sum(axis=2), np.ones((10,2))), "Transition probabilities should sum to 1"

value_iteration(hw1.P, hw1.R, gamma, n_iter=20)

         iter |  max|V-Vprev| | # chg actions |          V[0]
-------------------------------------------------------------
            1 |             1 |           N/A |             0
            2 |           0.9 |             2 |             0
            3 |          0.81 |             1 |             0
            4 |         0.729 |             1 |             0
            5 |        0.6561 |             1 |             0
            6 |       0.59049 |             1 |             0
            7 |      0.531441 |             1 |             0
            8 |      0.478297 |             1 |             0
            9 |      0.430467 |             1 |             0
           10 |       0.38742 |             1 |       0.38742
           11 |      0.348678 |             0 |      0.736099
           12 |      0.313811 |             0 |       1.04991
           13 |       0.28243 |             0 |       1.33234
           14 |      0.254187 |             0 |       1.58653
        

## Part 2: Policy Iteration

The next task is to implement exact policy iteration (PI).

PI first initializes the policy $\pi_0(s)$, and then it performs the following two steps on the $n$th iteration, for $n=1,2,...$:
1. Compute state-action value function $Q^{\pi_{n-1}}(s,a)$ of policy $\pi_{n-1}$
2. Compute new policy $\pi_n(s) = \operatorname*{argmax}_a Q^{\pi_{n-1}}(s,a)$

We'll break step 1 into two parts.

### Problem 2a: state value function

First you'll write a function called `compute_vpi` that computes the state-value function $V^{\pi}$ for an arbitrary policy $\pi$.
Recall that $V^{\pi}$ satisfies the following linear equation:
$$V^{\pi}(s) = \sum_{s'} T(s,\pi(s),s')[ R(s,\pi(s),s') + \gamma V^{\pi}(s')]$$

Your function should take as inputs:

- pi: 1D array specifying action at every state (i.e., the policy)
- T: transition probabilities
- R: reward function
- gamma: discount

Your function should output:

- Vpi: 1D array specifying value function at each state, satisfying the Bellman equation above.

Your function's output should exactly match the provided output below.

In [21]:
pi0 = np.zeros(nS,dtype='i') # policy performs action 0 at every state
Vpi = hw1.compute_vpi(pi0, P_rand, R_rand, gamma)
print Vpi

[ 5.206217    5.15900351  5.01725926  4.76913715  5.03154609  5.06171323
  4.97964471  5.28555573  5.13320501  5.08988046]


### Problem 2b: state-action value function

Next, you'll write a function to compute the state-action value function $Q^{\pi}$, defined as follows
$$Q^{\pi}(s) = \sum_{s'} T(s,a,s')[ R(s,a,s') + \gamma V^{\pi}(s')]$$

Your function should take as inputs:

- pi: 1D array specifying action at every state (i.e., the policy)
- T: transition probabilities
- R: reward function
- gamma: discount

Your function should output: 

- Qpi, a 2d array with axes corresponding to (state, action), satisfying the equation above.

Your function's output should exactly match the output below.


In [22]:
hw1.compute_qpi(pi0, P_rand, R_rand, gamma)

array([[ 5.206217  ,  5.20238706],
       [ 5.15900351,  5.1664316 ],
       [ 5.01725926,  4.99211906],
       [ 4.76913715,  4.98080235],
       [ 5.03154609,  4.89448888],
       [ 5.06171323,  5.29418621],
       [ 4.97964471,  5.06868986],
       [ 5.28555573,  4.9156956 ],
       [ 5.13320501,  4.97736801],
       [ 5.08988046,  5.00511597]])

The following code runs the policy iteration algorithm. Your output should match the provided output.

In [23]:
def policy_iteration(P, R, gamma, n_iter):
    pi_prev = np.zeros(P.shape[0],dtype='i')
    
    print hw_utils.fmt_row(13, ["iter", "# chg actions", "Q[0,0]"], header=True)
    
    for i in xrange(n_iter):
        qpi = hw1.compute_qpi(pi_prev, P, R, gamma)
        pi = qpi.argmax(axis=1)
        print hw_utils.fmt_row(13, [i+1, (pi != pi_prev).sum(),  qpi[0,0]])
        pi_prev = pi
        
policy_iteration(T_rand, R_rand, gamma, 10)

         iter | # chg actions |        Q[0,0]
---------------------------------------------
            1 |             4 |       5.20622
            2 |             2 |       5.59042
            3 |             0 |        5.6255
            4 |             0 |        5.6255
            5 |             0 |        5.6255
            6 |             0 |        5.6255
            7 |             0 |        5.6255
            8 |             0 |        5.6255
            9 |             0 |        5.6255
           10 |             0 |        5.6255
